## *DISCLAIMER*
<p style="font-size:16px; color:#117d30;">
 By accessing this code, you acknowledge the code is made available for presentation and demonstration purposes only and that the code: (1) is not subject to SOC 1 and SOC 2 compliance audits; (2) is not designed or intended to be a substitute for the professional advice, diagnosis, treatment, or judgment of a certified financial services professional; (3) is not designed, intended or made available as a medical device; and (4) is not designed or intended to be a substitute for professional medical advice, diagnosis, treatment or judgement. Do not use this code to replace, substitute, or provide professional financial advice or judgment, or to replace, substitute or provide medical advice, diagnosis, treatment or judgement. You are solely responsible for ensuring the regulatory, legal, and/or contractual compliance of any use of the code, including obtaining any authorizations or consents, and any solution you choose to build that incorporates this code in whole or in part.
</p>

## Important – Do not use in production, for demonstration purposes only – please review the legal notices before continuing
 License agreement: https://github.com/microsoft/Azure-Analytics-and-AI-Engagement/blob/main/HealthCare/License.md 


## Legal Notices
This presentation, demonstration, and demonstration model are for informational purposes only. Microsoft makes no warranties, express or implied, in this presentation demonstration, and demonstration model. Nothing in this presentation, demonstration, or demonstration model modifies any of the terms and conditions of Microsoft’s written and signed agreements. This is not an offer and applicable terms and the information provided is subject to revision and may be changed at any time by Microsoft.

This presentation, demonstration, and/or demonstration model do not give you or your organization any license to any patents, trademarks, copyrights, or other intellectual property covering the subject matter in this presentation, demonstration, and demonstration model.

The information contained in this presentation, demonstration and demonstration model represent the current view of Microsoft on the issues discussed as of the date of presentation and/or demonstration, and the duration of your access to the demonstration model. Because Microsoft must respond to changing market conditions, it should not be interpreted to be a commitment on the part of Microsoft, and Microsoft cannot guarantee the accuracy of any information presented after the date of presentation and/or demonstration and for the duration of your access to the demonstration model.

No Microsoft technology, nor any of its component technologies, including the demonstration model, is intended or made available: (1) as a medical device; (2) for the diagnosis of disease or other conditions, or in the cure, mitigation, treatment or prevention of a disease or other conditions; or (3) as a substitute for the professional clinical advice, opinion, or judgment of a treating healthcare professional. Partners or customers are responsible for ensuring the regulatory compliance of any solution they build using Microsoft technologies.

© 2020 Microsoft Corporation. All rights reserved


## Please don't run / don't click "Run all" the notebook:
At the time of writing of this document, the current core limit is 200 cores per workspace and depending upon number of concurrent users, you may end up with core capacity being exceeded or maximum number of parallel jobs being exceeded error.    


## Load Data Into DataFrame


In [2]:
%%pyspark
df = spark.read.load('abfss://saphana-finance-data@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/FinanceSalesData.csv', format='csv', header=True)
display(df)

## Data Transformation

In [18]:
%%pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *

import numpy as np
pd_df = df.select("ProductKey"
      ,"SalesOrderNumber"
      ,"OrderQuantity"
      ,"UnitPrice"
      ,"TotalProductCost"
      ,"SalesAmount","TaxAmt").toPandas()
convert_dict = {'TotalProductCost': float,
                'OrderQuantity': int,'TaxAmt':float,'SalesAmount':float}
pd_df=pd_df.astype(convert_dict)                
pd_df["TotalSale"]=pd_df.TotalProductCost*pd_df.OrderQuantity
pd_df["Profit"]=(pd_df.SalesAmount*pd_df.OrderQuantity)-(pd_df.OrderQuantity*pd_df.TotalProductCost)+pd_df.TaxAmt

pd_df.groupby(['SalesOrderNumber'])['Profit'].sum()
#pd_df=pd_df.where(pd_df['address1_country']!='None')
#pd_df.first()
print(pd_df)

ProductKey SalesOrderNumber  OrderQuantity  ...  TaxAmt  TotalSale  Profit
0             477          SO74554              1  ...  0.3992     1.8663  3.5229
1             477          SO74554              1  ...  0.3992     1.8663  3.5229
2             477          SO74554              1  ...  0.3992     1.8663  3.5229
3             477          SO74554              1  ...  0.3992     1.8663  3.5229
4             477          SO74554              1  ...  0.3992     1.8663  3.5229
...           ...              ...            ...  ...     ...        ...     ...
109995        477          SO74813              1  ...  0.3992     1.8663  3.5229
109996        477          SO74813              1  ...  0.3992     1.8663  3.5229
109997        477          SO74813              1  ...  0.3992     1.8663  3.5229
109998        477          SO74813              1  ...  0.3992     1.8663  3.5229
109999        477          SO74813              1  ...  0.3992     1.8663  3.5229

[110000 rows x 9 colum

In [19]:
%%pyspark
df = spark.createDataFrame(pd_df)
df.show(5)

(df
.coalesce(1)
 .write
 .mode("overwrite")
 .option("header", "true")
 .format("com.databricks.spark.csv")
 .save('abfss://saphana-finance-data@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/SalesData'))


+----------+----------------+-------------+------------------+----------------+-----------+------+---------+------------------+
|ProductKey|SalesOrderNumber|OrderQuantity|         UnitPrice|TotalProductCost|SalesAmount|TaxAmt|TotalSale|            Profit|
+----------+----------------+-------------+------------------+----------------+-----------+------+---------+------------------+
|       477|         SO74554|            1|4.9900000000000002|          1.8663|       4.99|0.3992|   1.8663|3.5229000000000004|
|       477|         SO74554|            1|4.9900000000000002|          1.8663|       4.99|0.3992|   1.8663|3.5229000000000004|
|       477|         SO74554|            1|4.9900000000000002|          1.8663|       4.99|0.3992|   1.8663|3.5229000000000004|
|       477|         SO74554|            1|4.9900000000000002|          1.8663|       4.99|0.3992|   1.8663|3.5229000000000004|
|       477|         SO74554|            1|4.9900000000000002|          1.8663|       4.99|0.3992|   1.8